In [1]:
import pandas as pd

- Bring in data from google doc using pd.read_csv

In [2]:
sheet_url = 'https://docs.google.com/spreadsheets/d/16uKsJASRhSDoPXun8rUfvBP1MhmgNA4n-tYSLfIQIZU/edit#gid=1617472581'    

csv_export_url = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')

df = pd.read_csv(csv_export_url)
df.head()

,Appraised Value,Tax Assessment Value,Tax Assemsent vs. Appraised Value,Tax as Percentage of Appraised Value,Retail MAP Price,Valuation Method,Confidence Score,Retail Market,AVM (Appraisal Completed 4),Loan Status,Subject Property Type,Subject Property County,LTV,CLTV,Loan Purpose,Lien Position,Loan Term,Loan Program
0,"290,000","225,840.00","45,367.97",78%,0,Drive By / Limited Scope,,0,NaN,Loan Originated,1 Unit,Comal,51.724,51.724,Home Improvement,First Lien,20.0,RBFCU Home Equity Fixed Rate 20 Yr (First Lien)
1,"258,602","234,990.00","16,696.21",91%,"258,602.00",Automated Valuation Model,very high,95.65,2/7/2020,Loan Originated,PUD,Bexar,75.405,75.405,Home Improvement,First Lien,30.0,RBFCU Home Equity Fixed Rate 30 Yr (First Lien)
2,"190,299","171,250.00","13,469.68",90%,"190,299.00",Automated Valuation Model,very high,100,1/22/2020,Loan Originated,PUD,Bexar,20.315,79.996,Home Improvement,Second Lien,15.0,RBFCU Home Equity Fixed Rate 15 Yr (Second Lien)
3,"380,914","380,914.00",0.00,NaN,0,Tax Assessment,,0,NaN,Loan Originated,PUD,Bexar,15.454,89.735,Home Improvement,Second Lien,15.0,RBFCU Home Improvement Fixed Rate 15 Yr (Secon...
4,"366,800","167,490.00","140,933.45",46%,"410,172.00",1st Full Appraisal,high,97.35,2/5/2020,Loan Originated,PUD,Bandera,47.710,47.710,Home Improvement,First Lien,10.0,RBFCU Home Equity Fixed Rate 10 Yr (First Lien)


- Get an overview of the data

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1059 entries, 0 to 1058
Data columns (total 18 columns):
Appraised Value                         1058 non-null object
Tax Assessment Value                    1059 non-null object
Tax Assemsent vs. Appraised Value       1059 non-null object
Tax as Percentage of Appraised Value    555 non-null object
Retail MAP Price                        1058 non-null object
Valuation Method                        1058 non-null object
Confidence Score                        1057 non-null object
Retail Market                           1058 non-null object
AVM (Appraisal Completed 4)             138 non-null object
Loan Status                             1058 non-null object
Subject Property Type                   1058 non-null object
Subject Property County                 1058 non-null object
LTV                                     1058 non-null float64
CLTV                                    1058 non-null float64
Loan Purpose                           

In [4]:
df.isnull().sum()

Appraised Value                           1
Tax Assessment Value                      0
Tax Assemsent vs. Appraised Value         0
Tax as Percentage of Appraised Value    504
Retail MAP Price                          1
Valuation Method                          1
Confidence Score                          2
Retail Market                             1
AVM (Appraisal Completed 4)             921
Loan Status                               1
Subject Property Type                     1
Subject Property County                   1
LTV                                       1
CLTV                                      1
Loan Purpose                              1
Lien Position                             1
Loan Term                                 1
Loan Program                              1
dtype: int64

- There's lot's of nulls here. Need to decide how best to deal with them 
- Several of these data types that should be numbers are strings with commas and % signs in them

# Deliverables
- This is what RBFCU wants:
> - What is delta between TAV (tax assessed value) and appraised value. Express as a percentage. To say differently, are TAVs consistently below appraised value and by what percentage.
           - Same for AVM (Automated value Model) to TAV.
> - Is there a greater variance when reviewed by county.
> - Does confidence interval play an important part or is there any correlation ...you’ll notice on some there is a High, or Very High score?
> - Confidence score null values can be assumed to have had a less than high or very high score because they have an appraised value- it just means that someone was lazy and didnt put that in there

- It will be presented to the board that handles risk for RBFCU. The audience will not be data scientist so we will need to be able to communicate our findings visually and in a way that they can understand. I have not been given a timeline but we can discuss that together in our zoom.

In [5]:
# Get rid of special characters ", and %"
df['Appraised Value'] = df['Appraised Value'].str.replace(',', '')
df['Tax Assessment Value'] = df['Tax Assessment Value'].str.replace(',', '')
df['Tax Assemsent vs. Appraised Value'] = df['Tax Assemsent vs. Appraised Value'].str.replace(',', '')
df['Retail MAP Price'] = df['Retail MAP Price'].str.replace(',', '')
df['Tax as Percentage of Appraised Value'] = df['Tax as Percentage of Appraised Value'].str.replace('%', '')

In [6]:
df.head()

,Appraised Value,Tax Assessment Value,Tax Assemsent vs. Appraised Value,Tax as Percentage of Appraised Value,Retail MAP Price,Valuation Method,Confidence Score,Retail Market,AVM (Appraisal Completed 4),Loan Status,Subject Property Type,Subject Property County,LTV,CLTV,Loan Purpose,Lien Position,Loan Term,Loan Program
0,290000,225840.00,45367.97,78,0,Drive By / Limited Scope,,0,NaN,Loan Originated,1 Unit,Comal,51.724,51.724,Home Improvement,First Lien,20.0,RBFCU Home Equity Fixed Rate 20 Yr (First Lien)
1,258602,234990.00,16696.21,91,258602.00,Automated Valuation Model,very high,95.65,2/7/2020,Loan Originated,PUD,Bexar,75.405,75.405,Home Improvement,First Lien,30.0,RBFCU Home Equity Fixed Rate 30 Yr (First Lien)
2,190299,171250.00,13469.68,90,190299.00,Automated Valuation Model,very high,100,1/22/2020,Loan Originated,PUD,Bexar,20.315,79.996,Home Improvement,Second Lien,15.0,RBFCU Home Equity Fixed Rate 15 Yr (Second Lien)
3,380914,380914.00,0.00,NaN,0,Tax Assessment,,0,NaN,Loan Originated,PUD,Bexar,15.454,89.735,Home Improvement,Second Lien,15.0,RBFCU Home Improvement Fixed Rate 15 Yr (Secon...
4,366800,167490.00,140933.45,46,410172.00,1st Full Appraisal,high,97.35,2/5/2020,Loan Originated,PUD,Bandera,47.710,47.710,Home Improvement,First Lien,10.0,RBFCU Home Equity Fixed Rate 10 Yr (First Lien)


- These are the columns that should probably be treated like floats
> Appraised Value, Tax Assessment Value, Tax Assemsent vs. Appraised Value, Tax as Percentage of Appraised Value,	Retail MAP Price, Retail Market

Start by getting rid of the spaces in the column names so you can call them later

In [7]:
df.columns = df.columns.str.replace(" ", "_")
df.columns = df.columns.str.replace("(", "_")
df.columns = df.columns.str.replace(')', "_")
df.head()

,Appraised_Value,Tax_Assessment_Value,Tax_Assemsent_vs._Appraised_Value,Tax_as_Percentage_of_Appraised_Value,Retail_MAP_Price,Valuation_Method,Confidence_Score,Retail_Market,AVM__Appraisal_Completed_4_,Loan_Status,Subject_Property_Type,Subject_Property_County,LTV,CLTV,Loan_Purpose,Lien_Position,Loan_Term,Loan_Program
0,290000,225840.00,45367.97,78,0,Drive By / Limited Scope,,0,NaN,Loan Originated,1 Unit,Comal,51.724,51.724,Home Improvement,First Lien,20.0,RBFCU Home Equity Fixed Rate 20 Yr (First Lien)
1,258602,234990.00,16696.21,91,258602.00,Automated Valuation Model,very high,95.65,2/7/2020,Loan Originated,PUD,Bexar,75.405,75.405,Home Improvement,First Lien,30.0,RBFCU Home Equity Fixed Rate 30 Yr (First Lien)
2,190299,171250.00,13469.68,90,190299.00,Automated Valuation Model,very high,100,1/22/2020,Loan Originated,PUD,Bexar,20.315,79.996,Home Improvement,Second Lien,15.0,RBFCU Home Equity Fixed Rate 15 Yr (Second Lien)
3,380914,380914.00,0.00,NaN,0,Tax Assessment,,0,NaN,Loan Originated,PUD,Bexar,15.454,89.735,Home Improvement,Second Lien,15.0,RBFCU Home Improvement Fixed Rate 15 Yr (Secon...
4,366800,167490.00,140933.45,46,410172.00,1st Full Appraisal,high,97.35,2/5/2020,Loan Originated,PUD,Bandera,47.710,47.710,Home Improvement,First Lien,10.0,RBFCU Home Equity Fixed Rate 10 Yr (First Lien)


- Now change the datatypes

In [8]:
df['Appraised_Value'] = df['Appraised_Value'].astype(float)
# df['Tax_Assessment_Value'] = df['Tax_Assessment_Value'].astype(float)
# When I tried to change the Tax Assessment Value I got an error. Turns out someone input the string 'Average:' into this field. 
# Find which observation it first

In [9]:
filt = df['Tax_Assessment_Value'] == 'Average:'
df[filt]

,Appraised_Value,Tax_Assessment_Value,Tax_Assemsent_vs._Appraised_Value,Tax_as_Percentage_of_Appraised_Value,Retail_MAP_Price,Valuation_Method,Confidence_Score,Retail_Market,AVM__Appraisal_Completed_4_,Loan_Status,Subject_Property_Type,Subject_Property_County,LTV,CLTV,Loan_Purpose,Lien_Position,Loan_Term,Loan_Program
1058,NaN,Average:,25695.95,86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- This observation also has lot's of nans. I'm gonna drop it for now

In [10]:
df = df.drop([1058])
# Check to see if there's one less row
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1058 entries, 0 to 1057
Data columns (total 18 columns):
Appraised_Value                         1058 non-null float64
Tax_Assessment_Value                    1058 non-null object
Tax_Assemsent_vs._Appraised_Value       1058 non-null object
Tax_as_Percentage_of_Appraised_Value    554 non-null object
Retail_MAP_Price                        1058 non-null object
Valuation_Method                        1058 non-null object
Confidence_Score                        1057 non-null object
Retail_Market                           1058 non-null object
AVM__Appraisal_Completed_4_             138 non-null object
Loan_Status                             1058 non-null object
Subject_Property_Type                   1058 non-null object
Subject_Property_County                 1058 non-null object
LTV                                     1058 non-null float64
CLTV                                    1058 non-null float64
Loan_Purpose                          

- That row is gone. Now try to change the data type of that column again

In [11]:
df['Tax_Assessment_Value'] = df['Tax_Assessment_Value'].astype(float)
# It worked!
# Move on

In [12]:
df['Tax_Assemsent_vs_Appraised_Value'] = df['Tax_Assemsent_vs._Appraised_Value'].astype(float)
df['Tax_as_Percentage_of_Appraised_Value'] = df['Tax_as_Percentage_of_Appraised_Value'].astype(float)
df['Retail_MAP_Price'] = df['Retail_MAP_Price'].astype(float)
# df['Retail_Market'] = df['Retail_Market'].astype(float)
# Apparently there's a comma in one of the rows in 'Retail_Market'.
# Replace all the coommas with empty string, then turn it to a float

In [13]:
df['Retail_Market'] = df['Retail_Market'].str.replace(',', '')

In [14]:
df['Retail_Market'] = df['Retail_Market'].astype(float)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1058 entries, 0 to 1057
Data columns (total 19 columns):
Appraised_Value                         1058 non-null float64
Tax_Assessment_Value                    1058 non-null float64
Tax_Assemsent_vs._Appraised_Value       1058 non-null object
Tax_as_Percentage_of_Appraised_Value    554 non-null float64
Retail_MAP_Price                        1058 non-null float64
Valuation_Method                        1058 non-null object
Confidence_Score                        1057 non-null object
Retail_Market                           1058 non-null float64
AVM__Appraisal_Completed_4_             138 non-null object
Loan_Status                             1058 non-null object
Subject_Property_Type                   1058 non-null object
Subject_Property_County                 1058 non-null object
LTV                                     1058 non-null float64
CLTV                                    1058 non-null float64
Loan_Purpose                      

- Looks like all our data types are appropriate now